In [1]:
import requests 
import os
from datetime import datetime
import pandas

# DOWNLOAD DATA FROM API

In [4]:
data= ''
n=0

while True :

    ploads = {'api-key':'579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b','format':'csv',
    'offset':n,
    'limit':10,
    # 'filters':'city=Calcutta',
    }
    r=requests.get('https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69',params=ploads)
    n+=10
    data+=r.text
    if r.text == '':
        break

# WRITE DATA TO CSV

In [75]:

time = datetime.now()
text_file = open("Air-{}.csv".format(time), "w")
n = text_file.write(data)
text_file.close()

## clean data

In [76]:
df=pandas.read_csv("Air-{}.csv".format(time))
df=df.drop_duplicates()
df=df.drop(df.index[[10]])


In [77]:

df=df.drop('pollutant_unit',axis=1)
df.dtypes

id               object
country          object
state            object
city             object
station          object
last_update      object
pollutant_id     object
pollutant_min    object
pollutant_max    object
pollutant_avg    object
dtype: object

In [78]:
df= df.fillna(0.0001)

df['pollutant_min']=df['pollutant_min'].astype('float')
df['pollutant_max']=df['pollutant_max'].astype('float')
df['pollutant_avg']=df['pollutant_avg'].astype('float')

#replace nans and change datatype

# CLEANED CSV

In [106]:
df['state'].unique()

array(['Andhra_Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Delhi',
       'Gujarat', 'Haryana', 'Jharkhand', 'Karnataka', 'Kerala',
       'Madhya Pradesh', 'Maharashtra', 'Meghalaya', 'Mizoram',
       'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'TamilNadu',
       'Telangana', 'Tripura', 'Uttar_Pradesh', 'West_Bengal'],
      dtype=object)

In [79]:
df

,id,country,state,city,station,last_update,pollutant_id,pollutant_min,pollutant_max,pollutant_avg
0,1,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",03-12-2020 11:00:00,PM2.5,49.0000,339.0000,104.0000
1,2,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",03-12-2020 11:00:00,PM10,59.0000,185.0000,95.0000
2,3,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",03-12-2020 11:00:00,NO2,18.0000,62.0000,27.0000
3,4,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",03-12-2020 11:00:00,NH3,3.0000,6.0000,4.0000
4,5,India,Andhra_Pradesh,Amaravati,"Secretariat, Amaravati - APPCB",03-12-2020 11:00:00,SO2,5.0000,31.0000,20.0000
...,...,...,...,...,...,...,...,...,...,...
1701,1548,India,West_Bengal,Kolkata,"Victoria, Kolkata - WBPCB",03-12-2020 11:00:00,NO2,26.0000,117.0000,80.0000
1702,1549,India,West_Bengal,Kolkata,"Victoria, Kolkata - WBPCB",03-12-2020 11:00:00,NH3,0.0001,0.0001,0.0001
1703,1550,India,West_Bengal,Kolkata,"Victoria, Kolkata - WBPCB",03-12-2020 11:00:00,SO2,10.0000,49.0000,22.0000
1705,1551,India,West_Bengal,Kolkata,"Victoria, Kolkata - WBPCB",03-12-2020 11:00:00,CO,26.0000,123.0000,63.0000


# Data Visulization

In [132]:
import plotly.express as px
def generateScatter(filterr,choice):
    fig = px.scatter(df[df[filterr]==choice], x="pollutant_avg", y="station", color="pollutant_id",size='pollutant_max')
    fig.update_layout(
        title="Pollutants in - {} ".format(choice),
        legend_title="Pollutants",
        font=dict(
            family="Courier New, monospace",
            size=12,
    )
    )
    filename="outs/(SCATTER)Pollutants in - {} ".format(choice)
    fig.write_html("{}.html".format(filename))
    fig.show()
generateScatter('state','Delhi')
generateScatter('city','Chennai'),

(None,)

In [133]:
import plotly.graph_objects as go
def generatePiechart(filterr,choice):
    labels = df['pollutant_id'].unique()

    values = []

    datachunk=df[df[filterr]==choice]
    for i in labels:
        cc=datachunk[datachunk['pollutant_id']==i]
        values.append(cc['pollutant_avg'].mean())

    # Use `hole` to create a donut-like pie chart
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3,)])
    fig.update_layout(
    title="Pollutants in - {} ".format(choice),
    legend_title="Pollutants",
    font=dict(
        family="Courier New, monospace",
        size=12,
        )
        )
    filename="outs/(PIE)Pollutants in - {} ".format(choice)
    fig.write_html("{}.html".format(filename))
    fig.show()

generatePiechart('state','Uttar_Pradesh'),generatePiechart('city','Lucknow')

(None, None)

In [98]:
labels

array(['PM2.5', 'PM10', 'NO2', 'NH3', 'SO2', 'CO', 'OZONE'], dtype=object)